In [1]:
import time
import warnings
from ripser import ripser
import numpy as np
from numpy import inf
import matplotlib.pyplot as plt
from persim import plot_diagrams
import persim
import glob
from numpy import asarray
from PIL import Image
import os

from sklearn import cluster, datasets
from sklearn.preprocessing import StandardScaler
from itertools import cycle, islice
import random
from sklearn.datasets import load_breast_cancer

from sklearn.semi_supervised import LabelPropagation
from sklearn.semi_supervised import LabelSpreading
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.svm import SVC
import warnings

import umap

import pandas as pd

from scipy.spatial.distance import pdist, squareform

import stablerank.srank as sr

warnings.filterwarnings('ignore')

In [2]:
!wget https://unirioja-my.sharepoint.com/:x:/g/personal/adines_unirioja_es/Ec1NYYxE6WRLlC0uuyXPzcgBO7w6wZcxW4WHsObhYBrHmA?download=1 -O banknote.csv

--2024-01-28 22:26:43--  https://unirioja-my.sharepoint.com/:x:/g/personal/adines_unirioja_es/Ec1NYYxE6WRLlC0uuyXPzcgBO7w6wZcxW4WHsObhYBrHmA?download=1
Resolving unirioja-my.sharepoint.com (unirioja-my.sharepoint.com)... 13.107.138.10, 13.107.136.10
Connecting to unirioja-my.sharepoint.com (unirioja-my.sharepoint.com)|13.107.138.10|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /personal/adines_unirioja_es/Documents/banknote.csv?ga=1 [following]
--2024-01-28 22:26:43--  https://unirioja-my.sharepoint.com/personal/adines_unirioja_es/Documents/banknote.csv?ga=1
Reusing existing connection to unirioja-my.sharepoint.com:443.
HTTP request sent, awaiting response... 200 OK
Length: 45030 (44K) [application/octet-stream]
Saving to: 'banknote.csv'

     0K .......... .......... .......... .......... ...       100% 11,9M=0,004s

2024-01-28 22:26:43 (11,9 MB/s) - 'banknote.csv' saved [45030/45030]



In [3]:
def load_prima():
  df=pd.read_csv("banknote.csv", sep=',',header=None)
  df1 = df.iloc[:,:-1]
  df2=df.iloc[:,-1]
  return df1.values,df2.values

In [4]:
def preparar_dataset(data,target,n_labeled=25):
  unos=np.where(target==1)[0]
  ceros=np.where(target==0)[0]

  datos_uno=np.array(data)[unos]
  datos_cero=np.array(data)[ceros]

  unos_valid=int((len(list(unos))-n_labeled)*0.2)
  ceros_valid=int((len(list(ceros))-n_labeled)*0.2)

  random.seed(15)
  indices_unos=random.sample(list(unos),n_labeled+unos_valid)

  random.seed(10)
  indices_ceros=random.sample(list(ceros),n_labeled+ceros_valid)


  puntos_unos=np.array(data)[indices_unos[0:n_labeled]]
  puntos_ceros=np.array(data)[indices_ceros[0:n_labeled]]

  puntos_unos_valid=np.array(data)[indices_unos[n_labeled:]]
  puntos_ceros_valid=np.array(data)[indices_ceros[n_labeled:]]

  unlabeled_unos=list(set(unos)-set(indices_unos))
  unlabeled_ceros=list(set(ceros)-set(indices_ceros))
  unlabeled=unlabeled_unos+unlabeled_ceros
  X_unlabeled_unos=np.array(data)[unlabeled_unos]
  X_unlabeled_ceros=np.array(data)[unlabeled_ceros]

  return (puntos_ceros,puntos_unos,X_unlabeled_ceros,X_unlabeled_unos,puntos_ceros_valid,puntos_unos_valid)

In [5]:
def analizar_punto_stable_rank_th(punto,puntos_ceros,puntos_unos,srank_ceros,srank_unos,sample=None,contour=sr.standard_contour(),clustering_method="single",w_p=inf,w_q=1,reduced=True,p=2,th=0.8):
    d_ceros_mod = sr.Distance(squareform(pdist(np.array([punto.tolist()]+puntos_ceros.tolist()), "euclidean")))
    srank_ceros_mod=d_ceros_mod.get_h0sr(clustering_method=clustering_method)
    d_unos_mod = sr.Distance(squareform(pdist(np.array([punto.tolist()]+puntos_unos.tolist()), "euclidean")))
    srank_unos_mod=d_unos_mod.get_h0sr(clustering_method=clustering_method)
    
    distance_cero=abs(srank_ceros.lp_distance(srank_ceros_mod,p))
    distance_uno=abs(srank_unos.lp_distance(srank_unos_mod,p))
    
    distancia=distance_cero+distance_uno
    if max(distance_cero/distancia,distance_uno/distancia)<th:
        clase=-1
    elif distance_cero>distance_uno:
        clase=1
    else:
        clase=0
    return clase

In [6]:
def analizar_punto_stable_rank(punto,puntos_ceros,puntos_unos,srank_ceros,srank_unos,sample=None,contour=sr.standard_contour(),clustering_method="single",w_p=inf,w_q=1,reduced=True,p=2,th=0):
    d_ceros_mod = sr.Distance(squareform(pdist(np.array([punto.tolist()]+puntos_ceros.tolist()), "euclidean")))
    srank_ceros_mod=d_ceros_mod.get_h0sr(clustering_method=clustering_method)
    d_unos_mod = sr.Distance(squareform(pdist(np.array([punto.tolist()]+puntos_unos.tolist()), "euclidean")))
    srank_unos_mod=d_unos_mod.get_h0sr(clustering_method=clustering_method)
    
    distance_cero=abs(srank_ceros.lp_distance(srank_ceros_mod,p))
    distance_uno=abs(srank_unos.lp_distance(srank_unos_mod,p))
    
    clase=-1
    if distance_cero>distance_uno:
        clase=1
    else:
        clase=0
    return clase

In [7]:
def analizar_puntos(data,target,func,sample=None,contour=sr.standard_contour(),w_p=inf,w_q=1,reduced=True,p=2,th=0,clustering_method="single",reduccion=False):
  (puntos_ceros,puntos_unos,X_unlabeled_ceros,X_unlabeled_unos,X_unlabeled_ceros_valid,X_unlabeled_unos_valid)=preparar_dataset(data,target)
  embedding=data
  if reduccion==True:
    embedding=umap.UMAP(random_state=75).fit_transform(data)
  (puntos_ceros_umap,puntos_unos_umap,X_unlabeled_ceros_umap,X_unlabeled_unos_umap,X_unlabeled_ceros_valid_umap,X_unlabeled_unos_valid_umap)=preparar_dataset(embedding,target)


  bien=0
  mal=0
  dudoso=0

  puntos_ceros_final=puntos_ceros.tolist()
  puntos_unos_final=puntos_unos.tolist()
  puntos_dudosos_final=[]

  d_ceros = sr.Distance(squareform(pdist(puntos_ceros_umap, "euclidean")))
  srank_ceros=d_ceros.get_h0sr(clustering_method=clustering_method)
  d_unos = sr.Distance(squareform(pdist(puntos_unos_umap, "euclidean")))
  srank_unos=d_unos.get_h0sr(clustering_method=clustering_method)
    
  for i,punto in enumerate(X_unlabeled_unos_umap):
    clase=func(punto,puntos_ceros_umap,puntos_unos_umap,srank_ceros,srank_unos,sample,contour,clustering_method,w_p,w_q,reduced,p,th)
    if clase==1:
      bien=bien+1
      puntos_unos_final.append(X_unlabeled_unos[i])
    elif clase==0:
      mal=mal+1
      puntos_ceros_final.append(X_unlabeled_unos[i])
    else:
      dudoso=dudoso+1
      puntos_dudosos_final.append(X_unlabeled_unos[i])


  
  for i,punto in enumerate(X_unlabeled_ceros_umap):
    clase=func(punto,puntos_ceros_umap,puntos_unos_umap,srank_ceros,srank_unos,sample,contour,clustering_method,w_p,w_q,reduced,p,th)
    if clase==1:
      mal=mal+1
      puntos_unos_final.append(X_unlabeled_ceros[i])
    elif clase==0:
      bien=bien+1
      puntos_ceros_final.append(X_unlabeled_ceros[i])
    else:
      dudoso=dudoso+1
      puntos_dudosos_final.append(X_unlabeled_ceros[i])
  
  comprobar_accuracy(np.array(puntos_ceros_final),np.array(puntos_unos_final),X_unlabeled_ceros_valid,X_unlabeled_unos_valid)

  return (bien, mal,dudoso,np.array(puntos_ceros_final),np.array(puntos_unos_final),np.array(puntos_dudosos_final))

In [8]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

def comprobar_accuracy(puntos_ceros_final,puntos_unos_final,X_unlabeled_ceros_valid,X_unlabeled_unos_valid):
    datos=np.concatenate((puntos_ceros_final,puntos_unos_final),axis=0)
    labels=np.concatenate((np.zeros(puntos_ceros_final.shape[0]),np.ones(puntos_unos_final.shape[0])))

    modelSVMLineal = SVC(kernel="linear",probability=True,random_state=90)
    modelSVMLineal.fit(datos, labels)

    mal=np.count_nonzero(modelSVMLineal.predict(X_unlabeled_ceros_valid)!=0)
    mal=mal+np.count_nonzero(modelSVMLineal.predict(X_unlabeled_unos_valid)!=1)
    total=X_unlabeled_ceros_valid.shape[0]+X_unlabeled_unos_valid.shape[0]
    print("--------------Clasificador SVM--------------")
    resumen(total-mal,mal,0)
    print("")


    # datos=np.concatenate((puntos_ceros_final,puntos_unos_final),axis=0)
    # labels=np.concatenate((np.zeros(puntos_ceros_final.shape[0]),np.ones(puntos_unos_final.shape[0])))

    modelRandomForest = RandomForestClassifier(random_state=90)
    modelRandomForest.fit(datos, labels)

    mal=np.count_nonzero(modelRandomForest.predict(X_unlabeled_ceros_valid)!=0)
    mal=mal+np.count_nonzero(modelRandomForest.predict(X_unlabeled_unos_valid)!=1)
    total=X_unlabeled_ceros_valid.shape[0]+X_unlabeled_unos_valid.shape[0]
    print("-------------- Clasificador Random Forest--------------")
    resumen(total-mal,mal,0)
    print("")

In [9]:
def resumen(bien,mal,dudosos):
  print('-----------------RESUMEN------------------')
  print('Bien anotados '+str(bien))
  print('Mal anotados '+str(mal))
  print('Sin anotar '+str(dudosos))
  print('')
  if bien+mal!=0:
    print('Porcentaje correcto '+str(bien/(bien+mal)))
  else:
    print('Porcentaje correcto 0')
  print('Porcentaje anotado '+str((bien+mal)/(bien+mal+dudosos)))

# Standard unreduced stable rank

## No threshold

In [10]:
(data,target) = load_prima()
(bien,mal,dudoso,pcero,puno,pdudoso)=analizar_puntos(data,target,analizar_punto_stable_rank)
resumen(bien,mal,dudoso)

--------------Clasificador SVM--------------
-----------------RESUMEN------------------
Bien anotados 257
Mal anotados 7
Sin anotar 0

Porcentaje correcto 0.9734848484848485
Porcentaje anotado 1.0

-------------- Clasificador Random Forest--------------
-----------------RESUMEN------------------
Bien anotados 257
Mal anotados 7
Sin anotar 0

Porcentaje correcto 0.9734848484848485
Porcentaje anotado 1.0

-----------------RESUMEN------------------
Bien anotados 1017
Mal anotados 41
Sin anotar 0

Porcentaje correcto 0.9612476370510397
Porcentaje anotado 1.0


## Threshold 0.8

In [11]:
(data,target) = load_prima()
(bien,mal,dudoso,pcero,puno,pdudoso)=analizar_puntos(data,target,analizar_punto_stable_rank_th,th=0.8)
resumen(bien,mal,dudoso)

--------------Clasificador SVM--------------
-----------------RESUMEN------------------
Bien anotados 256
Mal anotados 8
Sin anotar 0

Porcentaje correcto 0.9696969696969697
Porcentaje anotado 1.0

-------------- Clasificador Random Forest--------------
-----------------RESUMEN------------------
Bien anotados 234
Mal anotados 30
Sin anotar 0

Porcentaje correcto 0.8863636363636364
Porcentaje anotado 1.0

-----------------RESUMEN------------------
Bien anotados 20
Mal anotados 0
Sin anotar 1038

Porcentaje correcto 1.0
Porcentaje anotado 0.01890359168241966


## Threshold 0.6

In [12]:
(data,target) = load_prima()
(bien,mal,dudoso,pcero,puno,pdudoso)=analizar_puntos(data,target,analizar_punto_stable_rank_th,th=0.6)
resumen(bien,mal,dudoso)

--------------Clasificador SVM--------------
-----------------RESUMEN------------------
Bien anotados 257
Mal anotados 7
Sin anotar 0

Porcentaje correcto 0.9734848484848485
Porcentaje anotado 1.0

-------------- Clasificador Random Forest--------------
-----------------RESUMEN------------------
Bien anotados 238
Mal anotados 26
Sin anotar 0

Porcentaje correcto 0.9015151515151515
Porcentaje anotado 1.0

-----------------RESUMEN------------------
Bien anotados 548
Mal anotados 0
Sin anotar 510

Porcentaje correcto 1.0
Porcentaje anotado 0.5179584120982986


## Threshold 0.4

In [13]:
(data,target) = load_prima()
(bien,mal,dudoso,pcero,puno,pdudoso)=analizar_puntos(data,target,analizar_punto_stable_rank_th,th=0.4)
resumen(bien,mal,dudoso)

--------------Clasificador SVM--------------
-----------------RESUMEN------------------
Bien anotados 257
Mal anotados 7
Sin anotar 0

Porcentaje correcto 0.9734848484848485
Porcentaje anotado 1.0

-------------- Clasificador Random Forest--------------
-----------------RESUMEN------------------
Bien anotados 257
Mal anotados 7
Sin anotar 0

Porcentaje correcto 0.9734848484848485
Porcentaje anotado 1.0

-----------------RESUMEN------------------
Bien anotados 1017
Mal anotados 41
Sin anotar 0

Porcentaje correcto 0.9612476370510397
Porcentaje anotado 1.0


## Threshold 0.2

In [14]:
(data,target) = load_prima()
(bien,mal,dudoso,pcero,puno,pdudoso)=analizar_puntos(data,target,analizar_punto_stable_rank_th,th=0.2)
resumen(bien,mal,dudoso)

--------------Clasificador SVM--------------
-----------------RESUMEN------------------
Bien anotados 257
Mal anotados 7
Sin anotar 0

Porcentaje correcto 0.9734848484848485
Porcentaje anotado 1.0

-------------- Clasificador Random Forest--------------
-----------------RESUMEN------------------
Bien anotados 257
Mal anotados 7
Sin anotar 0

Porcentaje correcto 0.9734848484848485
Porcentaje anotado 1.0

-----------------RESUMEN------------------
Bien anotados 1017
Mal anotados 41
Sin anotar 0

Porcentaje correcto 0.9612476370510397
Porcentaje anotado 1.0


## UMAP no threshold

In [15]:
(data,target) = load_prima()
(bien,mal,dudoso,pcero,puno,pdudoso)=analizar_puntos(data,target,analizar_punto_stable_rank,reduccion=True)
resumen(bien,mal,dudoso)

--------------Clasificador SVM--------------
-----------------RESUMEN------------------
Bien anotados 255
Mal anotados 9
Sin anotar 0

Porcentaje correcto 0.9659090909090909
Porcentaje anotado 1.0

-------------- Clasificador Random Forest--------------
-----------------RESUMEN------------------
Bien anotados 244
Mal anotados 20
Sin anotar 0

Porcentaje correcto 0.9242424242424242
Porcentaje anotado 1.0

-----------------RESUMEN------------------
Bien anotados 972
Mal anotados 86
Sin anotar 0

Porcentaje correcto 0.9187145557655955
Porcentaje anotado 1.0


## UMAP threshold 0.8

In [16]:
(data,target) = load_prima()
(bien,mal,dudoso,pcero,puno,pdudoso)=analizar_puntos(data,target,analizar_punto_stable_rank_th,th=0.8,reduccion=True)
resumen(bien,mal,dudoso)

--------------Clasificador SVM--------------
-----------------RESUMEN------------------
Bien anotados 257
Mal anotados 7
Sin anotar 0

Porcentaje correcto 0.9734848484848485
Porcentaje anotado 1.0

-------------- Clasificador Random Forest--------------
-----------------RESUMEN------------------
Bien anotados 238
Mal anotados 26
Sin anotar 0

Porcentaje correcto 0.9015151515151515
Porcentaje anotado 1.0

-----------------RESUMEN------------------
Bien anotados 433
Mal anotados 0
Sin anotar 625

Porcentaje correcto 1.0
Porcentaje anotado 0.40926275992438566


## UMAP threshold 0.6

In [17]:
(data,target) = load_prima()
(bien,mal,dudoso,pcero,puno,pdudoso)=analizar_puntos(data,target,analizar_punto_stable_rank_th,th=0.6,reduccion=True)
resumen(bien,mal,dudoso)

--------------Clasificador SVM--------------
-----------------RESUMEN------------------
Bien anotados 262
Mal anotados 2
Sin anotar 0

Porcentaje correcto 0.9924242424242424
Porcentaje anotado 1.0

-------------- Clasificador Random Forest--------------
-----------------RESUMEN------------------
Bien anotados 253
Mal anotados 11
Sin anotar 0

Porcentaje correcto 0.9583333333333334
Porcentaje anotado 1.0

-----------------RESUMEN------------------
Bien anotados 938
Mal anotados 0
Sin anotar 120

Porcentaje correcto 1.0
Porcentaje anotado 0.8865784499054821


## UMAP threshold 0.4

In [18]:
(data,target) = load_prima()
(bien,mal,dudoso,pcero,puno,pdudoso)=analizar_puntos(data,target,analizar_punto_stable_rank_th,th=0.4,reduccion=True)
resumen(bien,mal,dudoso)

--------------Clasificador SVM--------------
-----------------RESUMEN------------------
Bien anotados 255
Mal anotados 9
Sin anotar 0

Porcentaje correcto 0.9659090909090909
Porcentaje anotado 1.0

-------------- Clasificador Random Forest--------------
-----------------RESUMEN------------------
Bien anotados 248
Mal anotados 16
Sin anotar 0

Porcentaje correcto 0.9393939393939394
Porcentaje anotado 1.0

-----------------RESUMEN------------------
Bien anotados 987
Mal anotados 71
Sin anotar 0

Porcentaje correcto 0.9328922495274102
Porcentaje anotado 1.0


## UMAP threshold 0.2

In [19]:
(data,target) = load_prima()
(bien,mal,dudoso,pcero,puno,pdudoso)=analizar_puntos(data,target,analizar_punto_stable_rank_th,th=0.2,reduccion=True)
resumen(bien,mal,dudoso)

--------------Clasificador SVM--------------
-----------------RESUMEN------------------
Bien anotados 257
Mal anotados 7
Sin anotar 0

Porcentaje correcto 0.9734848484848485
Porcentaje anotado 1.0

-------------- Clasificador Random Forest--------------
-----------------RESUMEN------------------
Bien anotados 252
Mal anotados 12
Sin anotar 0

Porcentaje correcto 0.9545454545454546
Porcentaje anotado 1.0

-----------------RESUMEN------------------
Bien anotados 1027
Mal anotados 31
Sin anotar 0

Porcentaje correcto 0.9706994328922496
Porcentaje anotado 1.0
